In [180]:
import yfinance as yf
import pandas as pd
import numpy as np

In [181]:
list=["HDFCBANK.NS", "ICICIBANK.NS", "AXISBANK.NS", "KOTAKBANK.NS", "SBIN.NS", "INDUSINDBK.NS", "BANKBARODA.NS",
     "FEDERALBNK.NS", "IDFCFIRSTB.NS", "PNB.NS"]

In [182]:
ticker1 = "AXISBANK.NS"

# Download 3 months of historical data using the daily time frame
data1 = yf.download(ticker1, start='2017-01-01', end='2022-01-01')[['Adj Close']]
len(data1)

arr = [[0 for i in range(len(data1))] for j in range(len(list))]

[*********************100%***********************]  1 of 1 completed


In [183]:
for i in range(len(list)):
    ticker=list[i]
    data=yf.download(ticker, start='2017-01-01', end='2022-01-01')[['Adj Close']]
    df_=pd.DataFrame(data)
    arr[i]=df_.to_numpy().reshape(1,-1)[0]
#     print(len(arr[i]))
    
# print(arr)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [184]:
pairs=[] #X,Y
for i in range(len(list)):
    for j in range(i+1,len(list)):
        x=arr[i]
        y=arr[j]
        r_x=(1/len(x) + (np.mean(x)**2) / np.sum((x - np.mean(x))**2))
        r_y=(1/len(y) + (np.mean(y)**2) / np.sum((y - np.mean(y))**2))
        if(r_x<=r_y):
            pairs.append([i,j])
        else:
            pairs.append([j,i])
            
print(len(pairs))

45


In [185]:
import statsmodels.api as sm
from statsmodels.tsa.stattools import adfuller

for i in range(len(pairs)):
    x_arr=arr[pairs[i][0]]
    y_arr=arr[pairs[i][1]]
    
    # Perform linear regression
    X = sm.add_constant(x_arr)  # Add a constant column to x_arr
    model = sm.OLS(y_arr, X)
    results = model.fit()

    # Get the slope, intercept, and residuals
    slope = results.params[1]
    intercept = results.params[0]
    residuals = results.resid

    # Perform the ADF test on the residual array
    adf_result = adfuller(residuals)
    adf_statistic = adf_result[0]
    p_value = adf_result[1]
    critical_values = adf_result[4]

    # Calculate the variance of the residuals
    std_residuals = np.std(residuals)
    
    pairs[i].append(slope)
    pairs[i].append(intercept)
    pairs[i].append(p_value)
    pairs[i].append(std_residuals)

In [186]:
mini=1
idx=0
count=0
for i in range(len(pairs)):
    if(pairs[i][4]<0.005):
        count+=1
    if(pairs[i][4]<mini):
        mini=pairs[i][4]
        idx=i
print(pairs[idx])
ftr=pairs[idx][2]
print(count)

[3, 0, 0.7093809254721486, 109.20205968968452, 0.0003500099763229711, 62.56961229811674]
1


In [187]:
s1=pairs[idx][0]
s2=pairs[idx][1]

In [188]:
ticker1 = list[s1]

# Download 3 months of historical data using the daily time frame
data1 = yf.download(ticker1, start='2022-01-01', end='2023-01-01')[['Adj Close']]
df1=pd.DataFrame(data1)

ticker2 = list[s2]

# Download 3 months of historical data using the daily time frame
data2 = yf.download(ticker2, start='2022-01-01', end='2023-01-01')[['Adj Close']]
df2=pd.DataFrame(data2)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [189]:
df_com=pd.DataFrame()
df_com.drop(df_com.columns, axis=1, inplace=True)
df_com['1st']=df1
df_com['2nd']=df2
df_com['std_err']=0
df_com

,1st,2nd,std_err
Date,,,
2022-01-03,1823.352295,1485.152466,0
2022-01-04,1851.485352,1493.850464,0
2022-01-05,1920.993530,1529.326294,0
2022-01-06,1890.761719,1504.796265,0
2022-01-07,1903.104370,1515.351074,0
...,...,...,...
2022-12-26,1813.550049,1610.975464,0
2022-12-27,1820.900024,1612.606812,0
2022-12-28,1820.099976,1611.321533,0


In [190]:
from datetime import datetime, timedelta

for i in range(len(df_com)):
    curr_date=df_com.index[i]
    
    start_date = curr_date - pd.DateOffset(days=200)
    
    end_date = curr_date - pd.DateOffset(days=1)
    

    # Download 3 months of historical data using the daily time frame
    data1 = yf.download(ticker1, start=start_date, end=end_date)[['Adj Close']]
    df1=pd.DataFrame(data1)
    x_arr=df1.to_numpy().reshape(1,-1)[0]

    # Download 3 months of historical data using the daily time frame
    data2 = yf.download(ticker2, start=start_date, end=end_date)[['Adj Close']]
    df2=pd.DataFrame(data2)
    y_arr=df2.to_numpy().reshape(1,-1)[0]
    
    
    # Perform linear regression
    X = sm.add_constant(x_arr)  # Add a constant column to x_arr
    model = sm.OLS(y_arr, X)
    results = model.fit()

    # Get the slope, intercept, and residuals
    slope = results.params[1]
    intercept = results.params[0]
    residuals = results.resid

    # Perform the ADF test on the residual array
    adf_result = adfuller(residuals)
    adf_statistic = adf_result[0]
    p_value = adf_result[1]
    critical_values = adf_result[4]

    # Calculate the variance of the residuals
    std_residuals = np.std(residuals)
    
    x=df_com.loc[curr_date, '1st']
    y=df_com.loc[curr_date, '2nd']
    y_pred=slope*x+intercept
    resi=y-y_pred
    std_err=resi/std_residuals
    df_com.loc[curr_date, 'resi']=resi
    df_com.loc[curr_date, 'std_residuals']=std_residuals
    df_com.loc[curr_date, 'std_err']=std_err
    

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [191]:
df_com['action']=0
df_com['state']='chill'
df_com

,1st,2nd,std_err,resi,std_residuals,action,state
Date,,,,,,,
2022-01-03,1823.352295,1485.152466,0.004196,0.180114,42.922727,0,chill
2022-01-04,1851.485352,1493.850464,-0.003911,-0.167932,42.942365,0,chill
2022-01-05,1920.993530,1529.326294,0.319934,13.720768,42.886252,0,chill
2022-01-06,1890.761719,1504.796265,-0.035256,-1.506449,42.728303,0,chill
2022-01-07,1903.104370,1515.351074,0.120763,5.143086,42.588086,0,chill
...,...,...,...,...,...,...,...
2022-12-26,1813.550049,1610.975464,2.924305,181.055716,61.914106,0,chill
2022-12-27,1820.900024,1612.606812,2.856522,175.297448,61.367442,0,chill
2022-12-28,1820.099976,1611.321533,2.742765,172.633882,62.941544,0,chill


In [192]:
for i in range(len(df_com.index)):
    idx=df_com.index[i]
    x=df_com.loc[idx, 'std_err']
    state=df_com.loc[idx, 'state']
    if(idx==df_com.index[-1]):
        break
    nxt=df_com.index[i+1]
    
    if(state=='chill'):
        if(x>-3 and x<-2.5):
            df_com.loc[idx, 'action']='buy'
            df_com.loc[nxt, 'state']='after buy'
        elif(x<3 and x>2.5):
            df_com.loc[idx, 'action']='sell'
            df_com.loc[nxt, 'state']='after sell'
        else:
            df_com.loc[idx, 'action']='wait'
            df_com.loc[nxt, 'state']='chill'
    
    if(state=='after buy'):
        if(x>=-1 or x<=-3):
            df_com.loc[idx, 'action']='sell'
            df_com.loc[nxt, 'state']='chill'
        else:
            df_com.loc[idx, 'action']='wait'
            df_com.loc[nxt, 'state']='after buy'
            
    if(state=='after sell'):
        if(x<=1 or x>=3):
            df_com.loc[idx, 'action']='buy'
            df_com.loc[nxt, 'state']='chill'
        else:
            df_com.loc[idx, 'action']='wait'
            df_com.loc[nxt, 'state']='after sell'

In [193]:
df_com

,1st,2nd,std_err,resi,std_residuals,action,state
Date,,,,,,,
2022-01-03,1823.352295,1485.152466,0.004196,0.180114,42.922727,wait,chill
2022-01-04,1851.485352,1493.850464,-0.003911,-0.167932,42.942365,wait,chill
2022-01-05,1920.993530,1529.326294,0.319934,13.720768,42.886252,wait,chill
2022-01-06,1890.761719,1504.796265,-0.035256,-1.506449,42.728303,wait,chill
2022-01-07,1903.104370,1515.351074,0.120763,5.143086,42.588086,wait,chill
...,...,...,...,...,...,...,...
2022-12-26,1813.550049,1610.975464,2.924305,181.055716,61.914106,wait,after sell
2022-12-27,1820.900024,1612.606812,2.856522,175.297448,61.367442,wait,after sell
2022-12-28,1820.099976,1611.321533,2.742765,172.633882,62.941544,wait,after sell


In [194]:
df_trunk=df_com
for i in range(len(df_com)-1, -1, -1):
    if(df_trunk.loc[df_com.index[i], 'action']=='wait' and df_trunk.loc[df_com.index[i], 'state']=='chill'):
        break
    else:
        df_trunk.drop(df_com.index[i], inplace=True)
        
df_trunk

,1st,2nd,std_err,resi,std_residuals,action,state
Date,,,,,,,
2022-01-03,1823.352295,1485.152466,0.004196,0.180114,42.922727,wait,chill
2022-01-04,1851.485352,1493.850464,-0.003911,-0.167932,42.942365,wait,chill
2022-01-05,1920.993530,1529.326294,0.319934,13.720768,42.886252,wait,chill
2022-01-06,1890.761719,1504.796265,-0.035256,-1.506449,42.728303,wait,chill
2022-01-07,1903.104370,1515.351074,0.120763,5.143086,42.588086,wait,chill
...,...,...,...,...,...,...,...
2022-12-08,1897.000000,1601.138306,2.328115,127.510574,54.769882,wait,chill
2022-12-09,1888.050049,1612.557373,2.621377,145.014314,55.319898,sell,chill
2022-12-12,1865.750000,1625.113403,3.226869,169.469889,52.518368,buy,after sell


In [195]:
bought=[]
sold=[]
for i in range(len(df_trunk)):
    idx=df_trunk.index[i]
    if(df_trunk.loc[idx, 'action']=='buy'):
        bought.append(df_trunk.loc[idx].to_numpy())
    elif(df_trunk.loc[idx, 'action']=='sell'):
        sold.append(df_trunk.loc[idx].to_numpy())
        
returns=[]
for i in range(len(bought)):
    cp=bought[i][1]+ftr*sold[i][0]
    sp=ftr*bought[i][0]+sold[i][1]
    returns.append(sp/cp)
    
np.cumprod(returns)

array([1.03024629, 1.04404655, 1.04007315, 1.03011776])